In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
import json
from pprint import pprint
import pandas as pd
from shapely.geometry import Polygon
import shapely
from sklearn import linear_model
import sklearn
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
from scipy import optimize
import pysr
from pysr import PySRRegressor
from IPython.display import display, Math
import torch
from tqdm import tqdm
%matplotlib widget

In [ ]:
df = pd.read_csv(
    './data/world_population.csv')
df = df.sort_values(by=['World Population Percentage'],ascending=False)
df[:10]

In [ ]:
cNames = [el for el in df]
CountryNames = df['Country/Territory'].values

In [ ]:
name_index = {name_:ind for name_,ind in zip(CountryNames, range(len(df)))}

In [ ]:
years_names = cNames[12:4:-1]
years_ = np.array([int(el[:4]) for el in years_names],dtype=np.float64)
years_

In [ ]:
pops_ = np.zeros(shape=(len(df),len(years_names)))
for i in range(len(df)):
    pops_[i] = df.iloc[i][years_names].values

In [ ]:
fig,ax = plt.subplots()
for i in range(10):
    ax.plot(years_, pops_[i]/10**9,label=CountryNames[i])
ax.tick_params(axis='x', rotation=45)
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# get velocity per country

In [ ]:
velocities_ = []
r2_vec = []
for i in range(len(df)):
    x_ = years_.reshape(-1,1)
    y_ = pops_[i]
    reg = linear_model.Ridge(alpha=.5)
    reg.fit(x_,y_)
    velocities_.append(reg.coef_[0])
    predict_ = reg.predict(x_)
    r2_vec.append(r2_score(y_,predict_))

In [ ]:
velocities_[name_index['Russia']]

In [ ]:
# fig,ax = plt.subplots()
# for i in range(len(df)):
fig,ax = plt.subplots()
fig.set_size_inches(16,9)
cnt_ = 0
for name_,v_,r2 in zip(CountryNames,velocities_,r2_vec):
    if r2 < 0.9:
        ax.plot(years_, pops_[name_index[name_]],label=name_)
        cnt_ +=1
ax.tick_params(axis='x', rotation=45)
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
# ax.set_yscale('log')
print(cnt_,len(CountryNames))

In [ ]:
t_vec =np.copy(years_)

In [ ]:
total_N_per_year = np.zeros(shape=(len(t_vec),))
for i in range(len(t_vec)):
    total_N_per_year[i] = np.sum(pops_[:,i])

In [ ]:
x_ = [[el] for el in t_vec]


In [ ]:
y_ = [el for el in total_N_per_year]


In [ ]:
poly = PolynomialFeatures(degree=1, include_bias=False)
poly_features = poly.fit_transform(t_vec.reshape(-1, 1))

In [ ]:
fig,ax = plt.subplots()
fig.set_size_inches(10,4)
ax.plot(t_vec,total_N_per_year,label='real')
ax.set_title(r'$\Sigma$')
# C0 = 186*10**9
# T0 = 2007.0
# y_vec = C0/(T0-t_vec)
# ax.plot(t_vec, y_vec, label=r'$\frac{d}{dt}u = \frac{1}{C_0}u^2$')
# P_vec = 6.463/(1.0+11.926*np.exp(-0.097*(t_vec-1960.0)))*10**9
# ax.plot(t_vec, P_vec ,label =r'$sigma model$')
# ax.set_yscale('log')
reg = linear_model.Ridge(alpha=.5)
reg.fit(x_,y_)
lin_vec = reg.intercept_ + reg.coef_[0]*t_vec
ax.plot(t_vec, lin_vec ,label =r'$y={} \cdot 10^9 +{}\cdot 10^6 t$'.format(str(reg.intercept_/10**9)[:4],str(reg.coef_[0]/10**6)[:4]))
ax.legend()
pred_ = reg.predict(x_)
r2_score(y_,pred_)

In [ ]:
x_ = np.array([\
1000.0,
1750.0,
1800.0,
1850.0,
1900.0,
1950.0,
1955.0,
1960.0,
1965.0,
1970.0,
1975.0,
1980.0,
1985.0,
1990.0,
1995.0,
2000.0,
2005.0,
2013.0],np.float64)
y_ = np.array([\
400000.0,
800000.0,
1000000.0,
1262000.0,
1656000.0,
2518629.0,
2755823.0,
3021475.0,
3334874.0,
3692492.0,
4068109.0,
4434682.0,
4830979.0,
5263593.0,
5674380.0,
6070581.0,
6343628.0,
7162119.0
],dtype=np.float64)*1000

In [ ]:
dydt_ = np.zeros(shape=(len(y_)-1,))
for i in range(len(dydt_)):
    tau = x_[i+1]-x_[i]
    dydt_[i] = (y_[i+1]-y_[i])/tau

In [ ]:
fig,ax =plt.subplots()
ax.plot(x_,y_)
ax.set_yscale('log')

In [ ]:
fig,ax =plt.subplots()
ax.plot(x_[:-1],dydt_)
ax.set_yscale('log')
print(np.max(dydt_))

In [ ]:
a = torch.tensor([186.0,2015.0]).clone().detach().requires_grad_(True)
x_t = torch.tensor(x_).clone().detach().requires_grad_(False)
y_t = torch.tensor(y_).clone().detach().requires_grad_(False)
z = [a]
last_lr=0.01
optimizer = torch.optim.Adam(z, last_lr, [0.5, 0.7])
EPOCH = 100000
def func(x,params):
    return params[0]*10**9/(params[1]-x)
def loss(x_vec,y_vec,params):
    y_pred = func(x_vec, params)
    return  1.0/len(x_vec)*torch.sum(torch.square(torch.log(y_pred)-torch.log(y_vec)))
loss_vec = []
for i in tqdm(range(EPOCH)):
    if i % 30000==0 and i>0:
        last_lr = last_lr/10.0
        for g in optimizer.param_groups:
            g['lr'] = last_lr
    optimizer.zero_grad()
    loss_ = loss(x_t,y_t, a)  
    loss_for_plot = float(loss_.cpu().detach().numpy())
    loss_vec.append(loss_for_plot)
    loss_.backward()
    optimizer.step()
fig,ax = plt.subplots()
ax.plot(np.arange(EPOCH),loss_vec)
ax.set_yscale('log')
a = a.cpu().detach().numpy()
alpha = a[0]
T0 = a[1]
print(f'alpha {alpha} T0 {T0}')

In [ ]:
alpha

In [ ]:
# alpha_ = 279.86
# T0_=2048.0
# preds_ = alpha_*10**9/(T0_-x_)
preds_ = alpha*10**9/(T0-x_)


In [ ]:
fig,ax =plt.subplots()
fig.set_size_inches(16,9)
ax.plot(x_,y_,label='data')
ax.plot(x_, preds_,label='predict')
# ax.set_yscale('log')
ax.legend()